In [ ]:
import pandas as pd
from pathlib import Path
import pandas as pd
import numpy as np
from datetime import datetime
from tqdm import tqdm
from plotnine import *
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.api.types import CategoricalDtype

from enhance_ocod.price_paid_process import gazetteer_generator
path = '../data/processed_price_paid/price_paid_2024.parquet'
ONSPD_path = "../data/ocod_history_processed/OCOD_FULL_2017_03.parquet"
figures_folder = Path('../figures')

In [ ]:
outframe = gazetteer_generator()

In [ ]:
outframe.loc[outframe['building'].str.contains('hamlyn')]

In [ ]:
cleaned_ppd = pd.read_parquet(path)

cleaned_ppd

In [ ]:
test = cleaned_ppd.loc[cleaned_ppd['paon'].str.contains('wharf|building|apartment|house', na = False)].copy()
test['building']= test['paon'].str.split(',').str[0]
test.loc[:,['building', 'oa11cd', 'lsoa11cd', 'msoa11cd', 'lad11cd' ]]

In [ ]:
test.drop_duplicates()

In [ ]:
    dtype_dict = {
        "paon": "string",
        "street": "string",
        "locality": "string",
        "postcode": "string",
        "price": "int32",
        "date_of_transfer": "string",
        "district": "string",
        "property_type": CategoricalDtype(categories=["D", "S", "T", "F", "O"]),
        "ppd_category_type": CategoricalDtype(categories=["A", "B"]),
    }

    price_paid_headers = [
        "transaction_unique_identifier",
        "price",
        "date_of_transfer",
        "postcode",
        "property_type",
        "old_new",
        "duration",
        "paon",
        "saon",
        "street",
        "locality",
        "town",
        "district",
        "county",
        "ppd_category_type",
        "record_status",
    ]

In [ ]:
test = pd.read_csv('/teamspace/studios/this_studio/enhance_ocod/data/price_paid_data/price_paid_complete_may_2025.csv', nrows = 10000,
            names=price_paid_headers,
            dtype=dtype_dict,)

In [ ]:
test['saon'].unique()

In [ ]:
test.loc[test['saon'].str.contains('UNIT', na = False)]

In [ ]:
test.loc[test['paon'].str.contains('FLAT', na = False)]